In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_rows = 3000
pd.options.display.max_columns = 3000

In [3]:
df = pd.read_csv('./datasets/backup2.csv')
df.head()

,title,time,sub,comments
0,Dad reflexes prevent crash.,2 hours ago,r/DadReflexes,249 comments
1,Quite literally choosing beggar,4 hours ago,r/ChoosingBeggars,631 comments
2,When your cat crashes his bicycle in his dream...,4 hours ago,r/funny,642 comments
3,Heatmap of numbers found at the end of Reddit ...,5 hours ago,r/dataisbeautiful,2612 comments
4,Hre you OPEN TODAY,5 hours ago,r/oldpeoplefacebook,313 comments


In [4]:
df.shape

(2525, 4)

In [5]:
# In the next few cells we clean up the 'comments' column, first reducing it to numeric strings
# and then oconverting those to ints. 
df['comments'] = df['comments'].apply(lambda x: x.replace('comments', ''))

In [6]:
df['comments'].value_counts()

2            121
1 comment     97
3             84
4             78
7             76
6             74
5             73
9             69
11            64
10            64
comment       63
14            59
8             57
12            49
15            42
18            38
13            37
20            31
22            30
16            29
23            28
17            28
21            25
37            25
31            24
28            23
19            23
24            22
26            21
35            20
27            20
25            16
29            16
32            15
30            15
42            14
36            14
34            14
64            13
38            13
39            13
43            12
45            11
33            11
40            11
57            10
68            10
50            10
65            10
41            10
79             9
61             9
60             9
77             9
47             9
80             9
53             9
51             8
59            

In [7]:
df['comments'] = df['comments'].apply(lambda x: x.replace('comment', ''))

In [8]:
df['comments'] = df['comments'].apply(lambda x: 0 if x == '' else x)

In [9]:
df['comments'] = df['comments'].apply(lambda x: int(x))

In [10]:
df.dtypes

title       object
time        object
sub         object
comments     int64
dtype: object

In [11]:
df['time'].value_counts()

6 hours ago       316
4 hours ago       309
5 hours ago       308
7 hours ago       279
3 hours ago       235
8 hours ago       199
2 hours ago       191
9 hours ago       117
10 hours ago       97
11 hours ago       74
1 hour ago         48
12 hours ago       43
13 hours ago       40
17 hours ago       36
15 hours ago       33
14 hours ago       28
16 hours ago       27
an hour ago        27
19 hours ago       24
18 hours ago       21
23 hours ago       19
20 hours ago       16
21 hours ago       12
22 hours ago       11
59 minutes ago      2
41 minutes ago      2
29 minutes ago      1
50 minutes ago      1
28 minutes ago      1
48 minutes ago      1
57 minutes ago      1
22 minutes ago      1
30 minutes ago      1
32 minutes ago      1
35 minutes ago      1
10 minutes ago      1
Name: time, dtype: int64

In [12]:
df['time'].apply(lambda x: type(x)).value_counts()

<type 'str'>      2524
<type 'float'>       1
Name: time, dtype: int64

In [13]:
df[df['time'].isnull() == True]

,title,time,sub,comments
1259,Turns out AB+ isn't always all that selfish,NaN,r/coolguides,174


In [14]:
df.isnull().sum()

title       0
time        1
sub         0
comments    0
dtype: int64

In [15]:
# Converting the 'time' column to floats which represent the number of hours a post has been up. 
df['time'] = df['time'].apply(lambda x: float(str(x)[0:2])/60 if 'minutes' in str(x)\
                 else float(str(x)[:-10]) if 'hours' in str(x) else x)

In [16]:
# Getting the '1 hour' ones, which weren't caught in the last cell. 
df['time'] = df['time'].apply(lambda x: 1 if 'hour' in str(x) else x)

In [17]:
df.dtypes

title        object
time        float64
sub          object
comments      int64
dtype: object

In [18]:
# Dropping rows with null values (which is just 1 row). 
df = df.dropna()

In [19]:
df.describe()

,time,comments
count,2524.000000,2524.000000
mean,6.845094,126.415214
std,4.405695,568.840644
min,0.166667,0.000000
25%,4.000000,7.000000
50%,6.000000,21.000000
75%,8.000000,73.000000
max,23.000000,17273.000000


In [20]:
# There distribution of the number of comments per post is highly skewed, 
# so the median might not be a good cutoff point for our classification. 
# Inspecting the proportion of posts that received 100 or more comments, 
# we see that it is just under 20%. This seems like it could be a reasonable cutoff.
len(df[df['comments'] >= 100])/float(len(df))

0.19730586370839936

In [21]:
# Creating a column called "H/L" (High/Low) where posts with 100 or more comments are classified as 'High'. 
df['H/L'] = df['comments'].apply(lambda x: 'Low' if x < 100 else 'High')

In [22]:
df.head()

,title,time,sub,comments,H/L
0,Dad reflexes prevent crash.,2.0,r/DadReflexes,249,High
1,Quite literally choosing beggar,4.0,r/ChoosingBeggars,631,High
2,When your cat crashes his bicycle in his dream...,4.0,r/funny,642,High
3,Heatmap of numbers found at the end of Reddit ...,5.0,r/dataisbeautiful,2612,High
4,Hre you OPEN TODAY,5.0,r/oldpeoplefacebook,313,High


In [23]:
df['H/L'].value_counts()

Low     2026
High     498
Name: H/L, dtype: int64

In [24]:
# Creating a new column - 'CpH' (Comments per Hour). 
df['CpH'] = df['comments']/df['time']

In [25]:
df.describe()

,time,comments,CpH
count,2524.000000,2524.000000,2524.000000
mean,6.845094,126.415214,17.321115
std,4.405695,568.840644,62.392326
min,0.166667,0.000000,0.000000
25%,4.000000,7.000000,1.375000
50%,6.000000,21.000000,4.000000
75%,8.000000,73.000000,11.428571
max,23.000000,17273.000000,1107.500000


In [26]:
# On second thought, maybe it's better to classify for a good rate - more than 10 comments per hour.
# About 28% of posts achieved that.
len(df[df['CpH'] >= 10])/float(len(df))

0.2777337559429477

In [27]:
# Adjusting the 'H/L' column accordingly. 
df['H/L'] = df['CpH'].apply(lambda x: 'Low' if x < 10 else 'High')

In [28]:
df.head()

,title,time,sub,comments,H/L,CpH
0,Dad reflexes prevent crash.,2.0,r/DadReflexes,249,High,124.50
1,Quite literally choosing beggar,4.0,r/ChoosingBeggars,631,High,157.75
2,When your cat crashes his bicycle in his dream...,4.0,r/funny,642,High,160.50
3,Heatmap of numbers found at the end of Reddit ...,5.0,r/dataisbeautiful,2612,High,522.40
4,Hre you OPEN TODAY,5.0,r/oldpeoplefacebook,313,High,62.60


In [29]:
# Reordering the columns to put the target at the end. 
cols = ['title', 'time', 'sub', 'comments', 'CpH', 'H/L']
df = df[cols]

In [30]:
df.head()

,title,time,sub,comments,CpH,H/L
0,Dad reflexes prevent crash.,2.0,r/DadReflexes,249,124.50,High
1,Quite literally choosing beggar,4.0,r/ChoosingBeggars,631,157.75,High
2,When your cat crashes his bicycle in his dream...,4.0,r/funny,642,160.50,High
3,Heatmap of numbers found at the end of Reddit ...,5.0,r/dataisbeautiful,2612,522.40,High
4,Hre you OPEN TODAY,5.0,r/oldpeoplefacebook,313,62.60,High


In [31]:
from sklearn.feature_extraction.text import CountVectorizer

# Instantiating a CountVectorizer to preprocess for NLP. An ngram range of 1-3 seems reasonable.
cvec = CountVectorizer(strip_accents='unicode', stop_words='english', ngram_range=(1,3), decode_error='replace')
cvec_df = cvec.fit(df['title'])

In [32]:
# Instantiating a new CountVectorizer with a maximum of 2000 features. 
# This was done because the models ran extremely slowly with the 23,000+ features of the original countvectorizer. 
# Although I made these after I first ran the model, I inserted them here for the sake of organization. 

cvec_2000 = CountVectorizer(strip_accents='unicode', stop_words='english',\
                            ngram_range=(1,3), decode_error='replace', max_features=2000)
cvec_2000_df = cvec_2000.fit(df['title'])

In [33]:
# The countvectorizer with only 2000 features did not get good results. 
# This is an attempt to find a good middle ground between speed and quality. 

cvec_5000 = CountVectorizer(strip_accents='unicode', stop_words='english',\
                            ngram_range=(1,3), decode_error='replace', max_features=5000)
cvec_5000_df = cvec_5000.fit(df['title'])

In [34]:
df['title']

0                             Dad reflexes prevent crash.
1                         Quite literally choosing beggar
2       When your cat crashes his bicycle in his dream...
3       Heatmap of numbers found at the end of Reddit ...
4                                      Hre you OPEN TODAY
5                       A tower of giraffes out for a run
6            My friend playing Mario Odyssey during class
7       2 years of saving $150 a month bought me this ...
8       This redwood tree stump is older than the Roma...
9                                   Sartre Day Night Live
10                               IBM Ball Head typewriter
11                   These traffic lights in the Ukraine.
12      Psychedelic mushrooms reduce authoritarianism ...
13                                   The flaming flip off
14                           PsBattle: Prize Winning Cake
15      "I Put Fries in an Enclosed Bowl So They Steam...
16                                  Welcome to Metropolis
17      Legit 

In [35]:
# Transforming each of the 3 countvectorizers to a dataframe.
cvec_df  = pd.DataFrame(cvec.transform(df['title']).todense(),
             columns=cvec.get_feature_names())

In [36]:
cvec_2000_df  = pd.DataFrame(cvec_2000.transform(df['title']).todense(),
             columns=cvec_2000.get_feature_names())

In [37]:
cvec_5000_df  = pd.DataFrame(cvec_5000.transform(df['title']).todense(),
             columns=cvec_5000.get_feature_names())

In [38]:
cvec_df.head(2)

,000,000 dodge,000 dodge demon,000 jobs,000 jobs close,000 medical,000 medical bills,000 socked,000 socked away,000 sqft,000 sqft facility,000 tons,000 tons corn,000 upvotes,000 upvotes the_donald,000 votes,000 year,000 year life,000 year mean,000 year old,000th,000th model,000th model year,01,01 2018,01 23,01 23 2018,03,06,06 20,08,08 pound,08 pound northern,08am,10,10 000,10 000 tons,10 000 votes,10 10,10 10 buy,10 10 guest,10 10 recommend,10 10 tasty,10 12,10 12 2018,10 14,10 14 days,10 22,10 22 build,10 buy,10 factories,10 files,10 game,10 game winning,10 guest,10 guest need,10 recommend,10 tasty,10 tasty recipe,10 years,100,100 000,100 000 dodge,100 000 socked,100 000 sqft,100 custom,100 custom frame,100 customer,100 customer work,100 days,100 doomed,100 epson,100 epson v800,100 joeesp,100 landed,100 landed regular,100 lets,100 percent,100 percent ________,100 pieces,100 pieces worldwide,100 pounds,100 pounds months,100 tons,100 tons radebaugh,100 year,100 year old,100 years,1000,1000 meters,1000 meters deep,100km,100km drive,100km drive fuel,101,101 don,101 don forget,1012,1024,1024 718,103,103 os,1080,1080 1080,1080 1349,1080 1349 polish,11,11 000,11 000 upvotes,11 43pm,11 43pm magnitude,11 available,11 available iphone,11 know,11 know identity,11 man,11 man career,11 released,11 year,11 year old,110,116,116 bjp,116 bjp mlas,1191,1191 1684,12,12 2018,12 22,12 22 years,12 circle,12 circle cannibalism,12 gain,12 previous,12 previous 13,12 strong,12 ways,12 ways divide,12 weeks,12 weeks tiny,12 years,12 years contracting,1200,1200 795,1200 795 oc,1280,1280 960,1280x,1280x 850,12am,12am est,12am est 24th,12l,12l nano,12l nano tank,13,13 03,13 half,13 half matches,13 inch,13 inch macbook,13 nominations,13 open,13 open issues,13 premios,13 premios oscar,13 seasons,13 seasons epkap,1335x2000,1349,1349 polish,1349 polish jw,1365x2048,13th,14,14 days,14 time,14 time marriage,14 year,14 year old,141,144p,144p senates,144p senates hd,14sh,14sh experimental,14sh experimental hovercraft,15,15 16,15 16 30,15 16 century,15 layout,15 layout signature,15 minimum,15 minimum wage,15 minutes,15 minutes netflix,15 years,15 years old,150,150 month,150 month bought,1500,1500 minimum,1500 minimum balance,151hrs,152,152 assault,152 assault gun,152 burning,152 burning distance,1536x1022,155,155 36,155 36 hrs,155lbs,155lbs 220lbs,155lbs 220lbs years,15hcp,15hcp play,15hcp play round,15mins,16,16 15,16 15 16,16 30,16 30 cet,16 century,16 mx5,16 mx5 club,160,160 000,160 000 year,1600,1600 1680,1620,1639,1639 oc,1639 oc 3264,1642,1642 meters,1642 meters deep,1680,1684,16nm,16nm wafers,16nm wafers tsmc,17,17 flying,17 flying fortresses,17 years,17 years sharing,1700s,178,1795,1795 1280,1795 1280 960,18,18 just,18 just corner,1850,1850 2590,1853,1853 targeting,1853 targeting norwegian,1889,19,19 45,19 45 gmt,19 year,19 year old,1909,1909 1910,1909 1910 1536x1022,1910,1910 1024,1910 1024 718,1910 1536x1022,1910 oldschoolcool,1918,1918 colourised,1920,1920 1080,1920s,1923,1927,1930,1930 2854,1930 2854 2202,1932,1938,1940s,1940s skyways,1940s skyways highways,1941,1942,1943,1943 2500x1825,1944,1944 3781,1944 3781 2862,1945,1946,1946 landed,1946 landed try,1947,1947 1080,1947 1080 1080,1958x2448,1958x2448 oc,1962,1962 movie,1962 movie tales,1964,1970,1974,1974 22,1974 22 year,1979,1979 aleksandar,1979 aleksandar savic,1980,1982,1983,1989,1989 era,1990,1992,1998,1998 time,1998 time thought,19th,19th fighter,19th fighter squadron,1a,1a challenge,1a challenge missouri,1uz,1uz v8,1uz v8 swap,20,20 000,20 000 year,20 97,20 97 complete,20 faces,20 faces counts,20 jahren,20 jahren im,20 jail,20 jail free,20 mins,20 mins looney,20 promo,20 years,20 years old,200,200 000th,200 000th model,200 btc,200 btc trading,2000,2000 41kg,2000 41kg haul,2000 best,2000x2500,2003,2007,2007 panorama,2007 panorama programme,200lbs,200lbs time,200lbs time years,2010,2010 worlds,2010 worlds impact,2011,2011 short,2011 short documentary,2013,2013 daniel,2013 daniel looking,2013 gli,

In [39]:
cvec_df.shape

(2524, 23263)

In [40]:
from sklearn.model_selection import train_test_split

#Creating a train/test split for each of the 3 countvectorizers. 

X = cvec_df
y = df['H/L']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [41]:
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(2019, 23263) (2019,)
(505, 23263) (505,)


In [42]:
X_2000 = cvec_2000_df

X_2000_train, X_2000_test, y_2000_train, y_2000_test = train_test_split(X_2000, y)

In [43]:
X_5000 = cvec_5000_df

X_5000_train, X_5000_test, y_5000_train, y_5000_test = train_test_split(X_5000, y)

In [44]:
# Trying a Random Forest with a gridsearch for the best 'max_features' parameter\
# (the max that the forest can see at each node). 
# Cross-validating with a stratified 5-fold split. 

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV

skf = StratifiedKFold(n_splits=5, shuffle=True)
rf = RandomForestClassifier(n_estimators=50, min_samples_split=2, min_samples_leaf=1, n_jobs=-1, verbose=2)

rf_params = {'max_features':['log2', 30, 65, 'sqrt', 250, 400]}

rf_grid = GridSearchCV(rf, param_grid=rf_params, cv=skf)
rf_grid.fit(X_2000_train, y_2000_train)
rf_scores = cross_val_score(rf, X_train, y_train, cv=skf)

building tree 1 of 50building tree 3 of 50building tree 2 of 50 


building tree 4 of 50
building tree 5 of 50
building tree 6 of 50 building tree 7 of 50

building tree 8 of 50
building tree 9 of 50building tree 10 of 50

building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 28 of 50
building tree 27 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.6s finished



building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


building tree 1 of 50building tree 2 of 50building tree 3 of 50building tree 4 of 50



building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50building tree 10 of 50

building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50building tree 20 of 50

building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50


...

[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   12.7s finished
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished


building tree 1 of 50building tree 2 of 50 building tree 4 of 50

building tree 3 of 50

building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    9.1s


building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   12.8s finished
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.1s finished


In [45]:
print rf_grid.best_params_
print rf_grid.best_score_
print rf_scores.mean()
print rf_scores.std()

{'max_features': 'log2'}
0.768092974115
0.797428936442
0.00852858386668


In [50]:
rf_grid.fit(X_5000_train, y_5000_train)

building tree 1 of 50
building tree 2 of 50
building tree 3 of 50building tree 4 of 50

building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50building tree 23 of 50

building tree 24 of 50
building tree 25 of 50 
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    0.8s


building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.2s finished
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.1s finished


building tree 1 of 50building tree 2 of 50

building tree 3 of 50building tree 4 of 50

building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50
building tree 14 of 50
building tree 15 of 50
 building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50 building tree 27 of 50

building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50building tree 32 of 50

building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50
building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 o

...

building tree 1 of 50building tree 3 of 50building tree 2 of 50building tree 4 of 50



building tree 5 of 50
building tree 6 of 50
building tree 7 of 50
building tree 8 of 50
building tree 9 of 50
building tree 10 of 50
building tree 11 of 50
building tree 12 of 50
building tree 13 of 50building tree 14 of 50

building tree 15 of 50
building tree 16 of 50
building tree 17 of 50
building tree 18 of 50
building tree 19 of 50
building tree 20 of 50
building tree 21 of 50
building tree 22 of 50
building tree 23 of 50
building tree 24 of 50
building tree 25 of 50
building tree 26 of 50
building tree 27 of 50
building tree 28 of 50
building tree 29 of 50
building tree 30 of 50
building tree 31 of 50
building tree 32 of 50
building tree 33 of 50
building tree 34 of 50
building tree 35 of 50
building tree 36 of 50
building tree 37 of 50
building tree 38 of 50
building tree 39 of 50
building tree 40 of 50


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    1.1s


building tree 41 of 50
building tree 42 of 50
building tree 43 of 50
building tree 44 of 50
building tree 45 of 50
building tree 46 of 50
building tree 47 of 50
building tree 48 of 50
building tree 49 of 50
building tree 50 of 50


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    1.6s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
            oob_score=False, random_state=None, verbose=2,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_features': ['log2', 30, 65, 'sqrt', 250, 400]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [51]:
y_train.value_counts()

Low     1464
High     555
Name: H/L, dtype: int64

In [52]:
y_train.value_counts()[0]/float(len(y_train))

0.72511144130757799

In [53]:
# Trying it on the 5000-feature countvectorizer. 

rf_grid.score(X_5000_test, y_5000_test)

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done  50 out of  50 | elapsed:    0.1s finished


0.81933438985736928

In [67]:
cvec_best_forest = rf_grid.best_estimator_
cvec_importances = pd.Series(cvec_best_forest.feature_importances_)
cvec_words = pd.Series(X_5000.columns)
cvec_importance_df = pd.concat([cvec_words, cvec_importances], axis=1).sort_values(1, ascending=False)
importance_df.rename(columns={0: 'Word', 1: 'Importance'}, inplace=True)

In [73]:
importance_df.head(10)

,Word,Importance
857,equality,0.004093
8,100 days,0.003998
2455,moment,0.003991
983,feelings,0.003772
4806,went,0.003760
3721,sent,0.003632
1301,going,0.003225
2171,love backyard,0.003219
6,100,0.003117
75,age,0.003097


In [74]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score

sapling = DecisionTreeClassifier(max_depth=3)
skf = StratifiedKFold(n_splits=5, shuffle=True)
abc = AdaBoostClassifier(base_estimator=sapling)
# ada_params = {"base_estimator__max_depth" : [1, 2, 3, 5, 8, 13, 21],
#               "n_estimators": [10, 25, 50]}
# The above line didn't seem to work so I decided to forego gridsearching for the best max depth of the underlying trees. 
ada_params = {"n_estimators": [10, 25, 50],
             'learning_rate':[0.1,0.5]}

ada_grid = GridSearchCV(abc, param_grid=ada_params, cv=skf, n_jobs=-1, verbose=2)
ada_grid.fit(X_2000_train, y_2000_train)

abc_scores = cross_val_score(abc, X_train, y_train, cv=skf)
print abc_scores
print abc_scores.mean()
print abc_scores.std()

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] n_estimators=10, learning_rate=0.1 ..............................
[CV] n_estimators=10, learning_rate=0.1 ..............................
[CV] n_estimators=10, learning_rate=0.1 ..............................
[CV] n_estimators=10, learning_rate=0.1 ..............................
[CV] ............... n_estimators=10, learning_rate=0.1, total=   2.5s
[CV] n_estimators=10, learning_rate=0.1 ..............................
[CV] ............... n_estimators=10, learning_rate=0.1, total=   2.6s
[CV] n_estimators=25, learning_rate=0.1 ..............................
[CV] ............... n_estimators=10, learning_rate=0.1, total=   2.6s
[CV] n_estimators=25, learning_rate=0.1 ..............................
[CV] ............... n_estimators=10, learning_rate=0.1, total=   2.7s
[CV] n_estimators=25, learning_rate=0.1 ..............................
[CV] ............... n_estimators=10, learning_rate=0.1, total=   2.6s
[CV] n_estimators

[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.0min finished


KeyboardInterrupt: 

In [ ]:
# Right around the baseline. Not good.
print ada_grid.best_params_
print ada_grid.best_score_

In [ ]:
# Trying with the more 'efficient' 2000 features. 
adaboost_2000 = ada_grid.fit(X_2000_train, y_2000_train)

In [ ]:
base_estimators = [DecisionTreeClassifier(max_depth=d) for d in [2,5,10,20]]
ada_grid = GridSearchCV(AdaBoostClassifier(), dict(base_estimator=base_estimators,\
                                               learning_rate=[.1,.3,.5], n_estimators=[10,25,50]), verbose=2)

In [ ]:
# Trying a Logistic Regression
from sklearn.linear_model import LogisticRegressionCV

logregcv = LogisticRegressionCV(Cs=np.logspace(-5,5,20), cv=skf, verbose=1)

In [ ]:
logregcv.fit(X_train, y_train)

In [ ]:
logregcv.get_params()

In [ ]:
# 81%, pretty good.
logregcv.score(X_test, y_test)

In [ ]:
coefs = pd.DataFrame(logregcv.coef_).T

In [ ]:
# But it seems like the coefs are in ascending order based on the index. That doesn't seem right.
coefs[0]

In [ ]:
# Why not Naive Bayes? 
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train, y_train)

In [ ]:
# But its performance is abysmal.
mnb.score(X_test, y_test)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


analyzer = SentimentIntensityAnalyzer()
for sentence in df.title.values[:10]:
    vs = analyzer.polarity_scores(sentence)
    print sentence
    print vs

In [ ]:
# Didn't really get anywhere analyzing titles, so...
# it's time to try analyzing subreddits. 
df['sub'].value_counts()

In [ ]:
# There are 1593 unique subreddits in our dataset.
df['sub'].nunique()

In [ ]:
bysub = df.groupby('sub')

In [ ]:
# This is just to see if the groupby worked. 
for g in bysub:
    print 'Subreddit:', g[0]
    print 'info:', g[1].head()['H/L']
    print '------------------------------------------'

In [ ]:
# Making a new DataFrame showing number of posts by subreddit.
newdf = df.groupby('sub').count().reset_index()

In [ ]:
# The new new DataFrame will only include subreddits with 3 or more posts. 
newnewdf = newdf[newdf['title'] > 2]
newnewdf.shape

In [ ]:
newnewdf = newnewdf.reset_index(drop=True)

In [ ]:
# "newnewdf" was based on the grouby and therefore each row was a subreddit. 
# "Good_df" will include all the posts from the subreddits of 'newnewdf'. 
good_df = df[df['sub'].isin(newnewdf['sub'])]

In [ ]:
good_df.shape

In [ ]:
good_df['sub'].nunique()

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
good_df.head()

In [ ]:
# The top 10 subreddits (with at least 3 posts) by comments per hour.
good_df.groupby('sub').mean().sort_values('CpH', ascending=False).iloc[:10, 2].plot(kind='bar')